# Verifica qualità delle etichette con Cleanlab

Cosa fa:
	•	Carica il dataset pesato (dataset_weighted.csv),
	•	Usa RandomForestClassifier + cross_val_predict,
	•	Applica Cleanlab per trovare potenziali errori di etichettatura,
	•	Esporta i casi sospetti in label_issues_detected.csv.

In [ ]:
# Se sei su Colab, esegui:
# !pip install cleanlab


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from cleanlab.classification import CleanLearning
from cleanlab.filter import find_label_issues

# Caricamento dataset (può essere pesato o base)
df = pd.read_csv("dataset_weighted.csv")

# Colonne dei sintomi
symptom_cols = [col for col in df.columns if "Symptom_" in col]


In [ ]:
# Features: vettore numerico da sintomi pesati
X = df[symptom_cols].values
y = df["Disease"].values


In [ ]:
# Classificatore + CleanLearning
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clean_model = CleanLearning(clf, seed=42)

# Trova etichette sospette
issues = find_label_issues(labels=y, pred_probs=cross_val_predict(clf, X, y, method='predict_proba'), return_indices_ranked_by='self_confidence')

# Mostra problemi trovati
df_issues = df.iloc[issues][["Disease"] + symptom_cols]
df_issues.head()


In [ ]:
df_issues.to_csv("label_issues_detected.csv", index=False)
print(f"Trovati {len(df_issues)} potenziali errori di etichettatura.")
